todo: create a folder original_data/ with the same path of this file
, download dataset https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification


# Data proprocessing

## loading the data

In [2]:
import os
import re
import speechbrain as sb
from speechbrain.processing.features import STFT, spectral_magnitude, Filterbank
import torch
import numpy as np

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [3]:
def get_music_file_names(root='original_data/genres_original', genre_list=[]):
    music_name_lists = []
    for genre in genre_list:
        music_name_lists.append(list(map(lambda x: genre + '/' + x,
                                         filter(lambda x: re.match(genre + '.(.*).wav', x),
                                                os.listdir(root + genre)))))
    return music_name_lists


In [4]:
music_name_lists = get_music_file_names('original_data/genres_original/', 
                                        ['rock', 'classical', 'metal', 'disco', 'blues', 'reggae', 'country', 'hiphop', 'jazz', 'pop'])

In [5]:
compute_STFT = STFT(
    sample_rate=22050, win_length=15, hop_length=20, n_fft=400
)

compute_fbanks = Filterbank(n_mels=40, sample_rate = 22050)

In [16]:
genre_num = len(music_name_lists)
features = None
labels = None
for label in range(genre_num):
    for music in music_name_lists[label]:
        waveform, sample_rate = torchaudio.load('original_data/genres_original/' + music)
        frag_num = int(waveform.shape[1] / (10 * sample_rate))
        #median = int(waveform.shape[1] / 2)
            # split wave into two minor fragments
        frag_list = [waveform[0:1, i * 10 * sample_rate: (i + 1) * 10 * sample_rate]
                        for i in range(0, frag_num)]
            # print(frag_list[0].shape)
        wave_fragment_features = [compute_fbanks(spectral_magnitude(compute_STFT(frag))) for frag in frag_list]
        if features is None:
            features = torch.cat(wave_fragment_features, dim=0)
        else:
            features = torch.cat((features, torch.cat(wave_fragment_features, dim=0)), dim=0)
        if labels is None:
            labels = torch.Tensor([label for i in range(frag_num)])
        else:
            labels = torch.cat((labels, torch.Tensor([label for i in range(frag_num)])), dim=0)
